In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from SubjectiveLogic import Hyperopinion, HyperopinionInterface
from SubjectiveLogic import Hyperopinion as W
from SubjectiveLogic.BeliefFusion import cumulative_fusion, averaging_fusion, weighted_fusion
from SubjectiveLogic.Trust import trust_discount_2e
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Callable

# Types

In [3]:
State = List['HyperopinionInterface']
TrustGraph = List[List['HyperopinionInterface']]
TrustArray = List['HyperopinionInterface']

# Conversion belief state to opinion

Unique hyperopinion such that belief state is the projected probability and base rate is uniform.

In [4]:
def opinion_from_belief_state(n: int) -> Hyperopinion:
    return Hyperopinion(2, [n, 1 - n]).maximize_uncertainty()

In [5]:
opinion_from_belief_state(0.5)

Hyperopinion(2, [0. 0.], [0.5 0.5])

In [6]:
opinion_from_belief_state(0.8)

Hyperopinion(2, [0.6 0. ], [0.5 0.5])

In [7]:
opinion_from_belief_state(0.2)

Hyperopinion(2, [0.  0.6], [0.5 0.5])

# Inicial state

## Uniform

In [8]:
def uniform_state(n: int) -> Hyperopinion:
    ans = []
    for i in range(n):
        ans.append(opinion_from_belief_state(i / (n - 1)))
    return(ans)

In [9]:
uniform_state(11)

[Hyperopinion(2, [0. 1.], [0.5 0.5]),
 Hyperopinion(2, [0.  0.8], [0.5 0.5]),
 Hyperopinion(2, [0.  0.6], [0.5 0.5]),
 Hyperopinion(2, [0.  0.4], [0.5 0.5]),
 Hyperopinion(2, [0.  0.2], [0.5 0.5]),
 Hyperopinion(2, [0. 0.], [0.5 0.5]),
 Hyperopinion(2, [0.2 0. ], [0.5 0.5]),
 Hyperopinion(2, [0.4 0. ], [0.5 0.5]),
 Hyperopinion(2, [0.6 0. ], [0.5 0.5]),
 Hyperopinion(2, [0.8 0. ], [0.5 0.5]),
 Hyperopinion(2, [1. 0.], [0.5 0.5])]

## Consensus

In [10]:
def consensus_state(n: int, b) -> State:
    ans = []
    for i in range(n):
        ans.append(opinion_from_belief_state(b))
    return(ans)

In [11]:
consensus_state(10, 0.5)

[Hyperopinion(2, [0. 0.], [0.5 0.5]),
 Hyperopinion(2, [0. 0.], [0.5 0.5]),
 Hyperopinion(2, [0. 0.], [0.5 0.5]),
 Hyperopinion(2, [0. 0.], [0.5 0.5]),
 Hyperopinion(2, [0. 0.], [0.5 0.5]),
 Hyperopinion(2, [0. 0.], [0.5 0.5]),
 Hyperopinion(2, [0. 0.], [0.5 0.5]),
 Hyperopinion(2, [0. 0.], [0.5 0.5]),
 Hyperopinion(2, [0. 0.], [0.5 0.5]),
 Hyperopinion(2, [0. 0.], [0.5 0.5])]

# Trust graph

## Clique

In [12]:
def clique(n: int, belief: float) -> TrustGraph:
    trust_graph = []
    for i in range(n):
        trust_graph.append([Hyperopinion(2, [belief, 1 - belief]) if i != j else None for j in range(n)])
    return trust_graph

In [13]:
trust_graph = clique(3, 0.5)
{i : trust_graph[i] for i in range(len(trust_graph))}

{0: [None,
  Hyperopinion(2, [0.5 0.5], [0.5 0.5]),
  Hyperopinion(2, [0.5 0.5], [0.5 0.5])],
 1: [Hyperopinion(2, [0.5 0.5], [0.5 0.5]),
  None,
  Hyperopinion(2, [0.5 0.5], [0.5 0.5])],
 2: [Hyperopinion(2, [0.5 0.5], [0.5 0.5]),
  Hyperopinion(2, [0.5 0.5], [0.5 0.5]),
  None]}

# Update

In [14]:
def update(
        fusion_operator: Callable[[int], State],
        truster_index: int,
        trust_array: TrustArray,
        state: State) -> HyperopinionInterface:
    truster_opinion = state[truster_index]
    discount_array = [trust_discount_2e(trust_array[trustee_index], state[trustee_index])
        for trustee_index in range(len(state)) if truster_index != trustee_index]
    new_opinion = fusion_operator([truster_opinion] + discount_array)
    return new_opinion

In [15]:
AX = opinion_from_belief_state(0.5)
BX = opinion_from_belief_state(1)
AB = Hyperopinion(2, [0.5, 0])
BA = Hyperopinion(2, [1, 0])
update(cumulative_fusion, 0, [AB, BA], [AX, BX]).P[0]

1.0

In [16]:
AX = opinion_from_belief_state(0.5)
print(AX)

k = 2
            
b({0}) = 0.0
b({1}) = 0.0
u = 1.0

a({0}) = 0.5
a({1}) = 0.5

P({0}) = 0.5
P({1}) = 0.5


In [17]:
BX = opinion_from_belief_state(1)
print(BX)

k = 2
            
b({0}) = 1.0
b({1}) = 0.0
u = 0.0

a({0}) = 0.5
a({1}) = 0.5

P({0}) = 1.0
P({1}) = 0.0


In [18]:
AB = Hyperopinion(2, [0.5, 0])
AB

Hyperopinion(2, [0.5 0. ], [0.5 0.5])

In [19]:
BA = Hyperopinion(2, [0.5, 0])
BA

Hyperopinion(2, [0.5 0. ], [0.5 0.5])

In [20]:
BAX = trust_discount_2e(BA, AX)
BAX

Hyperopinion(2, [0. 0.], [0.5 0.5])

In [21]:
averaging_fusion([BX, BAX]).P[0] # Why

1.0

In [22]:
cumulative_fusion([BX, BAX]).P[0]

1.0

# Original polarization paper

In [23]:
0.8 + 0.8 * 0.2

0.9600000000000001

In [26]:
AB = Hyperopinion(2, [0.5, 0.5])